3.1 Dimensão de datas

In [0]:
from pyspark.sql import functions as F

dates = (spark.table("silver_olist_orders")
  .select(F.to_date("order_purchase_timestamp").alias("dt"))
  .union(spark.table("silver_olist_orders").select(F.to_date("order_delivered_customer_date").alias("dt")))
  .union(spark.table("silver_olist_orders").select(F.to_date("order_estimated_delivery_date").alias("dt")))
  .dropna().distinct()
  .withColumn("date_key", F.date_format("dt", "yyyyMMdd").cast("int"))
  .withColumn("ano", F.year("dt"))
  .withColumn("mes", F.month("dt"))
  .withColumn("dia", F.dayofmonth("dt"))
  .withColumn("dia_semana", F.date_format("dt","E"))
  .withColumn("nome_mes", F.date_format("dt","MMMM"))
)

dates.write.format("delta").mode("overwrite").saveAsTable("gold_dim_data")


3.2 Dimensões simples

In [0]:
spark.table("bronze_olist_customers").write.format("delta").mode("overwrite").saveAsTable("gold_dim_cliente")
spark.table("bronze_olist_sellers").write.format("delta").mode("overwrite").saveAsTable("gold_dim_vendedor")
spark.table("silver_olist_products").write.format("delta").mode("overwrite").saveAsTable("gold_dim_produto")


3.3 Dim pedido com chaves de data

In [0]:
orders = (spark.table("silver_olist_orders")
  .withColumn("date_key_compra", F.date_format(F.to_date("order_purchase_timestamp"), "yyyyMMdd").cast("int"))
  .withColumn("date_key_entrega", F.date_format(F.to_date("order_delivered_customer_date"), "yyyyMMdd").cast("int"))
  .withColumn("date_key_estimativa", F.date_format(F.to_date("order_estimated_delivery_date"), "yyyyMMdd").cast("int"))
)

orders.write.format("delta").mode("overwrite").saveAsTable("gold_dim_pedido")


3.4 Fato principal (item do pedido)

In [0]:
from pyspark.sql import functions as F

items  = spark.table("silver_olist_order_items_enriched").alias("i")
orders = (spark.table("silver_olist_orders")
          .select("order_id","order_status","order_purchase_timestamp",
                  "order_delivered_customer_date","order_estimated_delivery_date")
          .alias("o"))

fact = (items
  .join(orders, on="order_id", how="left")
  .withColumn("date_key_compra",  F.date_format(F.to_date(F.col("o.order_purchase_timestamp")), "yyyyMMdd").cast("int"))
  .withColumn("date_key_entrega", F.date_format(F.to_date(F.col("o.order_delivered_customer_date")), "yyyyMMdd").cast("int"))
  .withColumn("date_key_estimativa", F.date_format(F.to_date(F.col("o.order_estimated_delivery_date")), "yyyyMMdd").cast("int"))
  .withColumn("total_item", (F.col("i.price") + F.col("i.freight_value")).cast("double"))
)

fact = fact.drop("order_purchase_timestamp", "order_delivered_customer_date", "order_estimated_delivery_date", "order_status")

fact.write.format("delta").mode("overwrite").saveAsTable("gold_fato_pedido_item")